# Sonification of Bleeding with Bank of Filters (Nature)

# First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [ ]:
import os
import copy
import threading
import matplotlib
import ipywidgets
import scipy.interpolate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import clear_output


In [ ]:
# %matplotlib inline

In [ ]:
#sonfn = [
#    "nature", "japan", "algomus", "eventearcons"]

#(son_nature_init, son_waterdrop_update, son_waterdrop_quit)
#sonfn = (son_waterdrop_init, son_waterdrop_update, son_waterdrop_quit)
#sonfn = (son_waterdrop_init, son_waterdrop_update, son_waterdrop_quit)
#sonfn = (son_waterdrop_init, son_waterdrop_update, son_waterdrop_quit)

## Load Data and Create Filter-Bank Signals

In [ ]:
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

delta_min = delta.min()
delta_max = delta.max()

volume_min = volume.min()
volume_max = volume.max()

print("dataset loaded:")
print(f"  delta:   min={delta_min:8}, max={delta_max:8.3}")
print(f"  volume:  min={volume_min:8}, max={volume_max:8}")

## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [ ]:
import sc3nb as scn
import time
sc = scn.startup()

In [ ]:
#%sc FreqScope(400, 300)
#%sc s.makeGui
#%sc s.scope

## Load Buffers and SynthDef

In [ ]:
b = sc.Buffer().load_file("samples/birds.wav")
r = sc.Buffer().load_file("samples/rain.wav")
w = sc.Buffer().load_file("samples/water-flow.wav")
c = sc.Buffer().load_file("samples/crickets.wav")
s = sc.Buffer().load_file("samples/sheeps.wav")
t = sc.Buffer().load_file("samples/thunder.wav")
e = sc.Buffer().load_file("samples/seaguls.wav")
f = sc.Buffer().load_file("samples/footstep.wav")
k = sc.Buffer().load_file("samples/kalimba.wav")
m = sc.Buffer().load_file("samples/motor.wav")
l = sc.Buffer().load_file("samples/bell.wav")

'''birds = b.bufnum
rain = r.bufnum
water_flow = w.bufnum
crickets = c.bufnum
sheeps = s.bufnum
thunder = t.bufnum
seaguls = s.bufnum
footstep = f.bufnum
kalimba = k.bufnum
motor = m.bufnum
bell = be.bufnum'''

In [ ]:
'''%%scv
b = Buffer.read(s, "samples/birds.wav");
r = Buffer.read(s, "samples/rain.wav");
w = Buffer.read(s, "samples/water-flow.wav");
c = Buffer.read(s, "samples/crickets.wav");
h = Buffer.read(s, "samples/sheeps.wav");
t = Buffer.read(s, "samples/thunder.wav");
e = Buffer.read(s, "samples/seaguls.wav");
f = Buffer.read(s, "samples/footstep.wav");
k = Buffer.read(s, "samples/kalimba.wav");
m = Buffer.read(s, "samples/motor.wav");
l = Buffer.read(s, "samples/bell.wav");
x = Buffer.read(s, "samples/car.wav");'''

In [ ]:
%%scv
SynthDef("pb-simple", { |out=0, bufnum=0, rate=1, pan=0, amp=0.3, loop=1, lgrt=2, lgamp = 0.5|
    var sig = PlayBuf.ar(2, bufnum, rate.lag(lgrt)*BufRateScale.kr(bufnum), loop, doneAction: 2);            
    Out.ar(out, Pan2.ar(sig, pan, amp.lag(lgamp)));
}).add;

In [ ]:
'''%%scv
SynthDef("pb-simple", { 
    |out=0, bufnum=0, loop=1, rate=1, trig=1, start=0, end, resetPos=0, pan=0, amp=0.3, 
    cf=1000, rq=1.0, lgrt=2, lgamp = 0.5|
    var sig, ptr;
    ptr = Phasor.ar(trig, rate.lag(lgrt)*BufRateScale.kr(bufnum), start, end, resetPos);
    sig = BufRd.ar(2, bufnum, ptr, loop: loop);
    sig = BPF.ar(sig, cf, rq);
    Out.ar(out, Pan2.ar(sig, pan, amp.lag(lgamp)));
}).add;'''

In [ ]:
#a = Synth.new(\bpf_motor, [\cf, 1000, \rq, 0.5, \rate, 0.5, \amp, 0.7]);
#a.set(\rate, 0.4)
#a.set(\rq, 0.7)
#a.set(\cf, 10000)

In [ ]:
#%sc ~snth = Synth.new("pb-simple", ["bufnum", r.bufnum, "rate", 1, "start", 0, "end", r.numFrames, "amp", 1])

In [ ]:
#%sc ~bell = Synth.new("pb-simple", ["bufnum", l.bufnum, "rate", 1, "start", 0, "end", l.numFrames, "resetPos", 10000, "amp", 1, "loop", 0])



In [ ]:
#%sc ~bell.free

In [ ]:
#%sc ~thunder.set("trig", 1)
#%scv postln(^al)

In [ ]:
#%sc s.freeAll

## Thread for Playing Data

In [ ]:
class Bloodplayer:
    
    def __init__(self, data, pulse_time=1, verbose=False):
        self.lock = threading.Lock()
        self.stopevent = threading.Event()
        self.callback_fn = None
        self.idx = 0
        self.data = data
        self.length = data.shape[0]
        self.verbose = verbose
        self.pulse_time = pulse_time
        
    #def __del__():
        # close plot window
        #pass
    
    def callback_fn_default(self, v):
        os.write(1, f"\r                       \r{v}".encode())
        
    def procfn(self):
        self.idx = 0
        while not self.stopevent.wait(0) and self.idx < self.length-1:
            v = self.data[self.idx]
            if self.verbose: 
                os.write(1, f"\r{self.idx}:{self.idx}                   ".encode())
            if callable(self.callback_fn):
                self.callback_fn(self)
            else:
                self.callback_fn_default(v)
            self.idx += 1
            time.sleep(self.pulse_time)
        print("done.")
    
    def set_callback(self, fn):
        self.callback_fn = fn
        
    def create_thread(self):
        threadname = "BloodPlayer-thread"
        # check first if it already exists
        if threadname in [t.name for t in threading.enumerate()]:
            print("create_thread: thread is already existing, stop first")
        else:
            self.stopevent.clear()
            self.producer = threading.Thread(name=threadname, target=self.procfn, args=[])
            self.producer.start()

    def stop_thread(self):
        self.stopevent.set()

In [ ]:
bloodplayer = Bloodplayer(delta)

## Plot Data

In [ ]:
%matplotlib

In [ ]:
# create figure
fig, ax = plt.subplots(1)  
mngr = plt.get_current_fig_manager(); 
mngr.window.setGeometry(840, 0, 600, 400)

# create axis
ax.clear()
plv0, = ax.plot([], [], "r-", lw=1.5)
plv1, = ax.plot([], [], "g-", lw=1.5)
plv2, = ax.plot([], [], "y-", lw=1.5)
plmarked, = ax.plot([], [], "r-", lw=1)
pldata, = ax.plot(delta, "-", lw=0.3)


def update_plot(self, t, x, v0, v1, v2): 
    global fig, ax, plmarked, plv1, plv2, plv3
    
    plmarked.set_data([t,t], [-10, 10])
    plv0.set_xdata(x[0:])
    plv0.set_ydata(v0[0:])
    plv1.set_xdata(x[0:])
    plv1.set_ydata(v1[0:])
    plv2.set_xdata(x[0:])
    plv2.set_ydata(v2[0:])
    ax.draw_artist(ax.patch)
    ax.draw_artist(plmarked)
    ax.draw_artist(pldata)
    ax.draw_artist(plv0)
    ax.draw_artist(plv1)
    ax.draw_artist(plv2)
    fig.canvas.update()


def onclick(event):
    global bloodplayer
    if event.dblclick:
        print(event.button, event.xdata)
        bloodplayer.idx = int(event.xdata)

connection_id = fig.canvas.mpl_connect('button_press_event', onclick)

## Sonification

In [ ]:
# Custom code for sonifications

v0 = [0,0]
v1 = [0,0]
v2 = [0,0]
vs0 = []
vs1 = []
vs2 = []
xs = []
tau = [5, 30, 120]
bn = [5000, 5001, 5002, 5003, 5004]
takt = [0,0]
amp = [0,0,0]
rate = [0,0,0]
pan = [0,0,0]
takt_rate = 50


def son_waterdrop_init():
    # initiall play buffer 
    #%sc ~water = Synth.new("pb-simple", ["bufnum", w.bufnum, "rate", 1, "start", 0, "end", w.numFrames, "amp", 0])
    #%sc ~seaguls = Synth.new("pb-simple", ["bufnum", e.bufnum, "rate", 1, "start", 0, "end", e.numFrames, "amp", 0])
    #%sc ~motor = Synth.new("pb-simple", ["bufnum", m.bufnum, "rate", 1, "start", 0, "end", m.numFrames, "amp", 0])
    sc.msg("/s_new", ["pb-simple", (bn[0]), 1, 1, "bufnum", w.bufnum, "rate", 1, "amp", 0])
    sc.msg("/s_new", ["pb-simple", (bn[1]), 1, 1, "bufnum", e.bufnum, "rate", 1, "amp", 0])
    sc.msg("/s_new", ["pb-simple", (bn[3]), 1, 1, "bufnum", m.bufnum, "rate", 1, "amp", 0])


def clock_event(takt):
    for t in range(1, takt+1): 
        sc.msg("/s_new", ["pb-simple", (5010+t), 1, 1, "bufnum", l.bufnum, "rate", 1, "amp", 0.7, "loop", 0])
        #%sc ~bell = Synth.new("pb-simple", ["bufnum", l.bufnum, "rate", 1, "start", 0, "end", l.numFrames-2, "amp", 1, "loop", 0])

def son_waterdrop(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp, rate, pan, takt_rate
    
    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]

    # normalize delta and volume
    nd = scn.linlin(delta_val, delta_min, delta_max, 0, 1)       
    nv = scn.linlin(volume_val, volume_min, volume_max, 0, 1)
    
    # tau 0 = 5 seconds
    refidx = max(self.idx-tau[0], 0)
    v0[0] = (volume.values[self.idx] - volume.values[refidx])/tau[0]
    amp[0] = scn.linlin(v0[0], 0, 3, 0.2, 0.9)
    sc.msg("/n_set", [bn[0], "rate", 1, "amp", amp[0], "lgrt", 2, "lgamp", 0.5])
    #%sc ~water.set("amp", ^amp[0])
    if v0[0] >= 1.5:
        rate[0] = scn.linlin(v0[0], 1.5, 3.2, 1.5, 3.5)
        sc.msg("/n_set", [bn[0], "rate", rate[0]])
        #%sc ~water.set("amp", ^rate[0])


    # tau 1 = 1 minute
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    if v1[0] < 0.5:
        amp[1] = v1[0]/2
        #%sc ~seaguls.set("amp", ^amp[1])
        sc.msg("/n_set", [bn[1], "rate", 1, "amp", amp[1]])

    if v1[0] >= 0.5:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1)
        sc.msg("/n_set", [bn[1], "rate", 1, "amp", amp[1]])
        #%sc ~seaguls.set("amp", ^amp[1])

        # event for exceeding threshold!
        if v1[1] < 0.7 and v1[0] >= 0.7:
            sc.msg("/s_new", ["pb-simple", bn[2], 1, 1, "bufnum", t.bufnum, "rate", 1, "amp", 1, "loop", 1])
            #%sc ~thunder = Synth.new("pb-simple", ["bufnum", t.bufnum, "rate", 1, "start", 0, "end", t.numFrames, "amp", 1])
        if v1[1] > 0.7 and v1[0] < 0.7:
            %sc ~thunder.free;
    
    v1[1] = v1[0]        
    
    

    # tau 2 = 5 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] >= 0.25:
        amp[2] = scn.linlin(v2[0], 0.25, 0.5, 0.01, 0.1)
        rate[2] = scn.linlin(v2[0], 0.25, 0.5, 0.6, 1)
        pan[2] = scn.linlin(v2[0], 0.25, 0.5, 1, 0)

        sc.msg("/n_set", [bn[3], "rate", rate[2], "amp", amp[2], "pan", pan[2]])
        #%sc ~motor.set("amp", ^amp[2], "rate", ^rate[2], "pan", ^pan[2])
        
        
    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    #print("takt is: " + str(takt))
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(takt[0])
    takt[1] = takt[0]
    
    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())
    
    # data for plots
    xs.append(self.idx)
    vs0.append(v0[0])
    vs1.append(v1[0])
    vs2.append(v2[0])
    
    update_plot(self, self.idx, xs, vs0, vs1, vs2)

def son_waterdrop_quit():
    #%sc s.freeAll
    sc.msg("/n_free", [bn[0]])
    sc.msg("/n_free", [bn[1]])
    sc.msg("/n_free", [bn[2]])
    sc.msg("/n_free", [bn[3]])
    
bloodplayer.set_callback(son_waterdrop)

## GUI

In [ ]:
# GUI
def start(b):
    global bloodplayer
    bloodplayer.create_thread()
    son_waterdrop_init()
    print("start")
b1 = ipywidgets.Button(description='Start') 
b1.on_click(start)

def stop(b):
    global bloodplayer
    print("stop")
    bloodplayer.stop_thread()
    son_waterdrop_quit()

b2 = ipywidgets.Button(description='Stop') 
b2.on_click(stop)

out = ipywidgets.Output()
ipywidgets.HBox([b1, b2, out])